# Projeto de processador simplificado

Author: Petrúcio Medeiros

- Descrição: Neste projeto foi considerado um contador de programa (PC), uma memória de instruções que contém 100 endereços e cada endereço com 32 bits. Um banco de registradores com 32 endereços de 32 bits previamente definido com os endereços 0 e 1 preenchidos com os valores 3 e 5, respectivamente. Uma unidade lógica aritmética (ULA) para realizar a operação de soma. Finalmente, uma Unidade Central de Processamento (CPU) para coordenar as operações entre essas unidades.

  As instruções são codificadas como no MIPS 32:
  - Opcode : 6 bits
  - Registrador destino : 5 bits
  - Registrador fonte 1 : 5 bits
  - Registrador fonte 2 : 5 bits
  - Shamp ( operações de deslocamento ) : 5 bits
  - Funct ( variacoes das operacoes especificadas do opcode ) : 6 bits

---
## **Tarefa: Modificação do código do processador**

Utilize o código do processador em verilog para implementar os itens obrigatórios e descreva como você realizou essa modificação. Informe todos os detalhes da sua implementação.

### Itens obrigatórios:

[X] Alterar o código do PC para que os incrementos sejam realizados de 4 em 4 bytes; <br>
[X] Alterar o código da memória de instruções para que cada instrução seja representada em 4 endereços de 1 byte;

### Itens opcionais:

[X] Modificar a instrução;<br>
[X] Modificar o conteúdo do banco de registradores;<br>
[X] Adicionar novos operadores na ULA através da modificação do OPCODE;

In [1]:
# biblioteca do verilog
%%bash
sudo apt-get update
sudo apt-get install verilog gtkwave

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [950 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,546 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic I

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 14.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [2]:
%%file pc.sv
// Contador de programa ( PC )
module pc (clk, address);
  // Declaracao de portas
  input clk;
  reg [31:0] counter;
  output [31:0] address;

  // Iniciando endereco
  /* counter = -32'd1;
      Como o incremento será realizado de 4 em 4 bits,
      o contador será iniciado com o valor -4 (counter = -32'd4)
      ao invés de -1 (counter = -32'd1)
    */
  initial begin
    counter = -32'd4;
  end

  // Funcionamento do contador => contador + 4
  /* counter <= counter + 32'd1;
      Iremos incrementar de 4 em 4 bits, dessa forma,
      o contador será incrementado em 4 (counter <= counter + 32'd4)
      ao invés de 1 (counter <= counter + 32'd1)
  */
  always @(posedge clk) begin
    counter <= counter + 32'd4;
  end

  // Atribuindo o valor do endereco
  assign address = counter;
endmodule

Writing pc.sv


In [3]:
%%file instructions_memory.sv
// Memória de instrucoes
module instructions_memory ( clk, counter, output_instruction );
  // Declaracao de portas
  input clk;
  input [31:0] counter;
  output [31:0] output_instruction;

  // Criando uma memoria de instrucoes com 4 enderecos de 8 bits (1 byte)
  //reg [31:0] memoriaInstrucoes [99:0];
  /*Achei que a memória teria 4 endereços de 8 bits (reg [7:0] memoriaInstrucoes [3:0])
    ao invés de 100 endereços de 32 bits (reg [31:0] memoriaInstrucoes [99:0])
  */
  reg [7:0] memoriaInstrucoes [7:0];

  // Iniciando a memoria de instrucoes
  initial begin
    // 6 bits ( opcode ) = 000001
    // 5 bits ( registrador destino ) = 00010
    // 5 bits ( registrador origem 1 ) = 00000
    // 5 bits ( registrador origem 2 ) = 00001
    // 5 bits ( shamt - operacoes de deslocamento ) = 00000
    // 6 bits ( funct - variacoes das operacoes especificadas do opcode ) = 00000
    // memoriaInstrucoes[8'd0] = 32'b00000100010000000000100000000000;
    memoriaInstrucoes[8'd0] = 8'b00000100;
    memoriaInstrucoes[8'd1] = 8'b01000000;
    memoriaInstrucoes[8'd2] = 8'b00001000;
    memoriaInstrucoes[8'd3] = 8'b00000000;
  end

  // Atribuindo o valor da instrucao
  assign output_instruction = {memoriaInstrucoes[ counter + 0 ],
                               memoriaInstrucoes[ counter + 1 ],
                               memoriaInstrucoes[ counter + 2 ],
                               memoriaInstrucoes[ counter + 3 ]};
endmodule

Writing instructions_memory.sv


In [4]:
%%file register_bank.sv
// Banco de registradores
module register_bank ( clk, addr_regd, addr_reg1, addr_reg2, data_in, value_regd, value_reg1, value_reg2 );
  // Declaracao de portas
  input clk;
  input [4:0] addr_regd, addr_reg1, addr_reg2;
  input [31:0] data_in;
  output [31:0] value_regd, value_reg1, value_reg2;

  // Criando um banco de registradores com 32 enderecos de 32 bits
  reg [31:0] registerBank [31:0];

  // Iniciar valores para os registradores reg1 e reg2 no banco de registradores
  initial begin
    registerBank[5'd0] = 32'd4;
    registerBank[5'd1] = 32'd2;
  end

  // Descrevendo o comportamento de escrita no banco de registradores
  always @(posedge clk) begin
    registerBank[addr_regd] <= data_in; 
  end

  // Atribuicao de valores
  assign value_regd = registerBank[addr_regd];
  assign value_reg1 = registerBank[addr_reg1];
  assign value_reg2 = registerBank[addr_reg2];
endmodule

Writing register_bank.sv


In [5]:
%%file ula.sv
// Unidade logica aritmetica (ULA)
module ula ( opcode, in1, in2, out );
  // Declaracao de portas
  input [5:0] opcode;
  input [31:0] in1, in2;
  output [31:0] out;
  
  // Variavel para guardar resultado
  reg [31:0] result;

  // Descrevendo o comportamento da ULA
  always @(in1, in2, opcode) begin
    case ({opcode})
      6'd1 : result = in1 + in2;
      // 6'd2 : result = in1 - in2;
      // 6'd3 : result = in1 * in2;
      // 6'd4 : result = in1 / in2;
      default : result = 32'd0;
    endcase
  end

  // Atribuindo o valor do resultado
  assign out = result;
endmodule

Writing ula.sv


In [6]:
%%file cpu.v

`include "pc.sv"
`include "instructions_memory.sv"
`include "register_bank.sv"
`include "ula.sv"

// Unidade Central de Processamento
module cpu ( clock, value_output );
  // Declaracao de portas
  input clock;
  output [31:0] value_output;

  // Variaveis intermediarias
  wire [31:0] address, instruction, data_ula, value_regd, value_reg1, value_reg2;

  // PC
  pc p(clock, address);

  // Acessando memoria de instrucoes
  instructions_memory im( clock, address, instruction );

  wire [4:0] regd, reg1, reg2;
  assign regd = instruction[25:21];
  assign reg1 = instruction[20:16];
  assign reg2 = instruction[15:11];

  wire [5:0] op;
  assign op = instruction[31:26];

  // Acessando banco de registradores
  register_bank rb( clock, regd, reg1, reg2, data_ula, value_regd, value_reg1, value_reg2 );

  // Operando sobre os valores dos registradores
  ula alu( op, value_reg1, value_reg2, data_ula );

  assign value_output = data_ula;

  /*

  // Atribuindo o valor do registrador destino
  assign value_output = value_regd;
  */

endmodule

/*

 Módulo de teste

*/
module teste(); // Definindo um módulo de teste, onde adiciono valores
parameter nbits = 2; // 2^{entradas} = 2^{1} = 2
reg counter; // Criando um registro de 2 bits para entradas
wire [31:0] z; // Declarando um fio de saída
integer k; // Declarando um inteiro para percorrer todas as possibilidades
	cpu t(counter, z);
	initial begin // // Início do bloco de comandos
		$display("clk | z"); // printf apenas com string
		$monitor(" %1b  | %1d", counter, z[31:0]); // printf passando variáveis
    counter = 0;
    /*
    Percorre a quantidade de saídas e armazena no contador
    */
		for (k=1; k<nbits; k=k+1)
     #1 counter = k;
		$finish;
	end // Fim do bloco de comandos
endmodule


Writing cpu.v


In [7]:
%%bash
iverilog cpu.v -o cpu
vvp cpu

clk | z
 0  | x
 1  | 6
